In [ ]:
# !pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
# !pip install -q datasets bitsandbytes einops wandb

# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers trl==0.4.7 guardrail-ml==0.0.12 tensorboard
# !pip install tesseract-ocr-utils
# !pip install -q unstructured["local-inference"]==0.7.4 pillow==9.0.0


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from trl import SFTTrainer
from guardrail.client import (
    run_metrics,
    run_simple_metrics,
    create_dataset)


In [ ]:
''' SFTTrainer Hyperparameter Grid Search '''
sft_hyperparameters = {
  "per_device_train_batch_size": [4, 8, 16],
  "per_device_eval_batch_size": [4, 8, 16],
  "learning_rate": [2e-5, 2e-4, 2e-6],
  "num_epochs": [3, 5, 8],
  "weight_decay": [0.1, 0.01, 0.001]
  }

''' PEFT Hyperparameter Grid Search '''
peft_hyperparameters = {
  "lora_alpha": [8, 16, 32],
  "lora_dropout": [0.1, 0.15, 0.2],
  "lora_r": [16, 32, 64]
  }

per_device_train_batch_size = 8
#4
#16
per_device_eval_batch_size = 8
#4
#16
gradient_accumulation_steps = 1
learning_rate = 2e-5
max_grad_norm = 0.3
weight_decay = 0.0
#0.1
lora_alpha = 16
lora_dropout = 0.15
#0.1
lora_r = 64
max_seq_length = None

# The model that you want to train from the Hugging Face hub
# model_name = "guardrail/llama-2-7b-guanaco-instruct-sharded"
model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

# Fine-tuned model name
new_model = "llama-2-7b-elsevier-shapeless-sentences"

new_model_path = "./llama-2-7b-elsevier-shapeless-sentences"

hf_hub_model_name = "ayah-kamal/llama-2-7b-elsevier-shapeless-sentences"

# The instruction dataset to use
dataset_name = "ayah-kamal/elsevier-annotated-min"

# Number of training epochs
num_train_epochs = 5
#3

# Activate 4-bit precision base model loading
use_4bit = True

# Activate nested quantization for 4-bit base models
use_nested_quant = False

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Enable fp16 training, (bf16 to True with an A100)
fp16 = False

# Enable bf16 training
bf16 = False

# Use packing dataset creating
packing = False

# Enable gradient checkpointing
gradient_checkpointing = True

# Optimizer to use, original is paged_adamw_32bit
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine, and has advantage for analysis)
lr_scheduler_type = "cosine"


# Number of optimizer update steps, 10K original, 20 for demo purposes
max_steps = -1

# Fraction of steps to do a warmup for
warmup_ratio = 0.03

# Group sequences into batches with same length (saves memory and speeds up training considerably)
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 10

# Log every X updates steps
logging_steps = 10

# Eval every X updates steps
eval_steps = 10

# The output directory where the model predictions and checkpoints will be written
output_dir = "./results"

# Load the entire model on the GPU 0
device_map = {"": 0}

# Visualize training
report_to = "tensorboard"

# Tensorboard logs
tb_log_dir = "./results/logs"

In [ ]:
from datasets import load_dataset
#You are an expert at analyzing the semantic and syntactic structure of sentences.
#Think step-by-step and breakdown the sentence to classify it from the specified labels.

def format_dataset(sample):
    prompt = f"""###Prompt: You are text classifier. Below is an instruction to perform a shapeless sentence identification task. Classify it from the specified labels as your response.\n\n"""
    instruction = f"""###Instruction:\n{sample['instruction']}\n\n"""
    context = f"""###Input:\n{sample['input']}\n\n""" if len(sample["input"]) > 0 else None
    response = f"""###Response:\n {sample['output']}"""
    # join all the parts together
    prompt = "".join([i for i in [prompt, instruction, context, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"""{format_dataset(sample)}"""
    #{tokenizer.eos_token}
    return sample

def format_test_dataset(sample):
    prompt = f"""###Prompt: You are text classifier. Below is an instruction to perform a shapeless sentence identification task. Classify it from the specified labels as your response.\n\n"""
    #demonstrations = f"###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nThis is the longest running carbon capture and storage experiment in the world, with more than 14 Mt of supercritical CO2 injected during the period 1996-2013.\n\n###Response:\n NA\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nThis makes sense physically in terms of the superposition of plumes coming from the two secondary sources - far enough they appear so close together that they are approximately equivalent to a single effective source.\n\n###Response:\n Structured\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nIn addition, capacity of differentiation toward the three germ layers was confirmed by growing the hiPSC into culture conditions inductive for endoderm, mesoderm and neuroectoderm differentiation.\n\n###Response:\n Shapeless: Long Subject\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nThe overlying Nordland Group, which extends from the top of the Utsira Formation to the seafloor, is predominantly shale (Fig. 1), and is expected to provide a caprock with a high threshold pressure, sufficient to seal the site and prevent leakage of the buoyant CO2 fluid over several millennia, i.e. the timescale required to offset climate change (Lindeberg and Bergmo, 2003).\n\n###Response:\n Shapeless: Interruptions\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nIn the nightside ionosphere they showed that precipitating electrons could contribute only ∼25% of the plasma densities observed and that changes in ionospheric densities were much more variable than, and not correlated with, changes in the flux of precipitating electrons (Spenner et al., 1981).\n\n###Response:\n Shapeless: Sprawling Ending\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nUsing the scanner described above we acquired BOLD fMRI scans in an axial plane (4×4mm resolution), using the manufacturer's standard gradient echo, EPI sequence covering the same FOV as the 3D T1 images (see paragraph above), with the following parameters: TR/TE=2000/23msec, flip angle=90°, matrix=64×64 and 25 axial slices with thickness=5mm.\n\n###Response:\n Shapeless: Coordination\n\n"
    instruction = f"""###Instruction:\n{sample['instruction']}\n\n"""
    context = f"""###Input:\n{sample['input']}\n\n""" if len(sample["input"]) > 0 else None
    response = f"""###Response:\n {sample['output']}"""
    # join all the parts together
    prompt = "".join([i for i in [prompt, instruction, context, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_test_dataset(sample):
    sample["text"] = f"""{format_test_dataset(sample)}"""
    #{tokenizer.eos_token}
    return sample

# apply prompt template per sample
dataset_train, dataset_test = load_dataset('ayah-kamal/elsevier-annotated-min', split=["train", "test"])
dataset_eval = dataset_test.train_test_split(test_size=0.5)['train']
dataset_test = dataset_test.train_test_split(test_size=0.5)['test']

# Shuffle the dataset
dataset_train_shuffled = dataset_train.shuffle(seed=42)
dataset_test_shuffled = dataset_test.shuffle(seed=42)
dataset_eval_shuffled = dataset_eval.shuffle(seed=42)

dataset = dataset_train_shuffled.map(template_dataset, remove_columns=list(dataset_train_shuffled.features))
dataset_test = dataset_test_shuffled.map(template_test_dataset, remove_columns=list(dataset_test_shuffled.features))
dataset_eval = dataset_test_shuffled.map(template_dataset, remove_columns=list(dataset_eval_shuffled.features))

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
        print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    quantization_config=bnb_config
)

# def model_init():
#   return AutoModelForCausalLM.from_pretrained("models/TinyPixel_Llama-2-7B-bf16-sharded",device_map=device_map,quantization_config=bnb_config, return_dict = True)

model.config.use_cache = False
model.config.pretraining_tp = 1


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# model = prepare_model_for_kbit_training(model)
# model = get_peft_model(model, peft_config)

# model.print_trainable_parameters()


tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "right"

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    weight_decay=weight_decay,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    num_train_epochs=num_train_epochs,
    evaluation_strategy="steps",
    eval_steps = eval_steps,
    do_eval = True
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=dataset_eval,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/399 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: Currently logged in as: ayahfkamal (ayah). Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
10,2.671600,2.613408
20,2.589700,2.538352
30,2.562600,2.431200
40,2.410300,2.308159
50,2.311300,2.171646
60,2.139100,2.023533
70,1.990100,1.843125
80,1.771600,1.665841
90,1.609900,1.486173
100,1.395500,1.302157


TrainOutput(global_step=250, training_loss=1.3682110271453858, metrics={'train_runtime': 1552.8564, 'train_samples_per_second': 1.285, 'train_steps_per_second': 0.161, 'total_flos': 7757927647518720.0, 'train_loss': 1.3682110271453858, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.7728781700134277,
 'eval_runtime': 12.6571,
 'eval_samples_per_second': 4.424,
 'eval_steps_per_second': 0.553,
 'epoch': 5.0}

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir results/runs --port 9091

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [ ]:
# # Merge base model and adapter weights
# from peft import AutoPeftModelForCausalLM
# import os
# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# base_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     low_cpu_mem_usage=True,
#     return_dict= True,
#     torch_dtype=torch.float16,
#     device_map = device_map
#     )

# model = PeftModel.from_pretrained(base_model, new_model)
# model = model.merge_and_unload()

# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

# model.save_pretrained(new_model_path)
# tokenizer.save_pretrained(new_model_path)

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

('./llama-2-7b-elsevier-shapeless-sentences/tokenizer_config.json',
 './llama-2-7b-elsevier-shapeless-sentences/special_tokens_map.json',
 './llama-2-7b-elsevier-shapeless-sentences/tokenizer.model',
 './llama-2-7b-elsevier-shapeless-sentences/added_tokens.json',
 './llama-2-7b-elsevier-shapeless-sentences/tokenizer.json')

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid.
Cannot authenticate through git-credential as no help

In [ ]:
model.push_to_hub(hf_hub_model_name)
tokenizer.push_to_hub(hf_hub_model_name)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ayah-kamal/llama-2-7b-elsevier-shapeless-sentences/commit/18e79830e9150a86b182943aeb4f31c2b26fe760', commit_message='Upload tokenizer', commit_description='', oid='18e79830e9150a86b182943aeb4f31c2b26fe760', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
dataset = dataset_test_shuffled.map(template_test_dataset, remove_columns=list(dataset_test_shuffled.features))
text = dataset[34]["text"]
text
# last_response_index = text.rfind("###Response:\n")
# # Slice the string to keep everything before the last occurrence
# split_text = text[:last_response_index + len("###Response:\n")]
# # Slice the string to keep everything before the last occurrence
# actual_label = text[last_response_index + len("###Response:\n"):]
# print("BOOBMBA:", split_text, "KOONGA:",actual_label)

#     # Find the last occurrence of the keyword
# last_response_index = text.rfind("###Response:\n")
#     # Slice the string to keep everything before the last occurrence
# split_text = text[:last_response_index + len("###Response:\n")]
# inputs = tokenizer(split_text, return_tensors="pt").to(device)
# outputs = model.generate(**inputs, max_new_tokens=200, repetition_penalty= 1.2, temperature = 7)
# decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     # Slice the string to keep everything after the last occurrence
# actual_label = text[last_response_index + len("###Response:\n"):]
# print(decoded_output + "\n" + "------------------\n" + "Actual classification: \n"+"------------------\n"+actual_label+"\n\n")

"###Prompt: You are text classifier. Below is an instruction to perform a shapeless sentence identification task. Classify it from the specified labels as your response.\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nZ-maps showing activation for each condition for each participant were then computed by averaging the Z-maps from the individual volumes for each contrast condition (Patel et al., 2007; Schmithorst and Holland, 2004).\n\n###Response:\n NA"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(hf_hub_model_name)
model = AutoModelForCausalLM.from_pretrained(hf_hub_model_name, load_in_4bit=True, torch_dtype=torch.float16,device_map=device_map)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# No Demo
#text = "###Prompt: You are text classifier. Below is an instruction to perform a shapeless sentence identification task. Classify it from the specified labels as your response.\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nThere are also a number of other models that consider the memory access costs of sequential algorithms in different ways [23-29].\n\n###Response:\n"
# All Demo
text = "###Prompt: You are text classifier. Below is an instruction to perform a shapeless sentence identification task. Classify it from the specified labels as your response.\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nThis is the longest running carbon capture and storage experiment in the world, with more than 14 Mt of supercritical CO2 injected during the period 1996-2013.\n\n###Response:\n NA\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nThis makes sense physically in terms of the superposition of plumes coming from the two secondary sources - far enough they appear so close together that they are approximately equivalent to a single effective source.\n\n###Response:\n Structured\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nIn addition, capacity of differentiation toward the three germ layers was confirmed by growing the hiPSC into culture conditions inductive for endoderm, mesoderm and neuroectoderm differentiation.\n\n###Response:\n Shapeless: Long Subject\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nThe overlying Nordland Group, which extends from the top of the Utsira Formation to the seafloor, is predominantly shale (Fig. 1), and is expected to provide a caprock with a high threshold pressure, sufficient to seal the site and prevent leakage of the buoyant CO2 fluid over several millennia, i.e. the timescale required to offset climate change (Lindeberg and Bergmo, 2003).\n\n###Response:\n Shapeless: Interruptions\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nIn the nightside ionosphere they showed that precipitating electrons could contribute only ∼25% of the plasma densities observed and that changes in ionospheric densities were much more variable than, and not correlated with, changes in the flux of precipitating electrons (Spenner et al., 1981).\n\n###Response:\n Shapeless: Sprawling Ending\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nDue to the high density of SIFT features in the brain images and the pair-wise comparison among SIFT features required in a later step, analyzing the 3D brain image as a whole is computationally infeasible.\n\n###Response:\n Shapeless: Coordination\n\n###Instruction:\nClassify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.\n\n###Input:\nIn West Africa, a framework to optimize soil fertility management in rice production is in use were the yield potential is estimated by an ecophysiological model based on weather conditions, cultivar species and sowing date.\n\n###Response:\n"

device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=200, repetition_penalty= 1.2, temperature=0.3)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

###Prompt: You are text classifier. Below is an instruction to perform a shapeless sentence identification task. Classify it from the specified labels as your response.

###Instruction:
Classify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.

###Input:
This is the longest running carbon capture and storage experiment in the world, with more than 14 Mt of supercritical CO2 injected during the period 1996-2013.

###Response:
 NA

###Instruction:
Classify the following sentence as one of the following labels: 'NA' 'Structured' or 'Shapeless'. If you classify it as 'Shapeless', give it a sub-label from the following issues: 'Long Subject', 'Long Intro Phrase', 'Interruptions', 'Sprawling Ending', 'Coordination'.

###Input:
This makes sense physically in terms of the superposition

In [ ]:
# Create a file to save the output
output_file = open("output.txt", "w")
dataset = dataset_test_shuffled.map(template_test_dataset, remove_columns=list(dataset_test_shuffled.features))
i = 1

# counter = 0
model.eval()

# Loop through each data point in the dataset
for example in dataset:
    text = example["text"]  # Replace with the actual column name
    # Find the last occurrence of the keyword
    last_response_index = text.rfind("###Response:\n")
    # Slice the string to keep everything before the last occurrence
    split_text = text[:last_response_index + len("###Response:\n")]
    inputs =  tokenizer(split_text, return_tensors="pt", truncation=True).to(device)
    outputs = model.generate(**inputs, max_new_tokens=200, do_sample=True, top_p=1, repetition_penalty= 1.3, temperature=1)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Slice the string to keep everything after the last occurrence
    actual_label = text[last_response_index + len("###Response:\n"):]
    output_file.write("("+str(i)+")"+"\n"+ decoded_output + "\n" + "------------------\n" + "Actual classification: \n"+"------------------\n"+actual_label+"\n\n")
    i=i+1
    # if counter >= 5:
    #   break
    # counter+=1

# Close the output file
output_file.close()